In [1]:
import numpy as np
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates


In [2]:

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose

In [9]:



####BODY POSE ESTIMATION 

# image = cv2.imread("/home/bundy/Pictures/Webcam/hand.jpg") 
# with mp_hands.Hands(
#     static_image_mode=True,
#     max_num_hands = 2,
#     min_detection_confidence=0.5
# ) as hands:
#     results = hands.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
#     if results.multi_hand_landmarks:
#         print((results.multi_hand_landmarks[0]))




pose_image = cv2.imread("/home/bundy/Pictures/Webcam/bodypose.jpg") 
# load image using cv2....and do processing.
# as opencv loads in BGR format by default, we want to show it in RGB.

def get_hands(pose_image):
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    ) as pose:
        handmarks = [21,19,15,17]
        max_x = 0
        max_y = 0
        min_x = 1000
        min_y = 1000

        pose_image = cv2.flip(pose_image,1)
        results = pose.process(cv2.cvtColor(pose_image,cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            pass
            # mp_drawing.draw_landmarks(
            #     pose_image,
            #     results.pose_landmarks,
            #     mp_pose.POSE_CONNECTIONS,
            #     landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            # )
            for index in handmarks:
                landmark = results.pose_landmarks.landmark[index]
                if landmark.x < min_x:
                    min_x = landmark.x
                elif landmark.x > max_x:
                    max_x = landmark.x
                if landmark.y < min_y:
                    min_y = landmark.y
                elif landmark.y > max_y:
                    max_y = landmark.y

            min_x  = int(min_x * pose_image.shape[1] / 1.2)
            max_x  = int(max_x * pose_image.shape[1] * 1.2) 
            min_y  = int(min_y * pose_image.shape[0] / 1.2) 
            max_y  = int(max_y * pose_image.shape[0] * 1.2) 
            cv2.rectangle(pose_image, (min_x,min_y), (max_x,max_y), (0, 255, 0), 2)
            pose_image = cv2.cvtColor(pose_image, cv2.COLOR_BGR2RGB)
            # pose_image = pose_image[min_y:max_y,min_x:max_x]


    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands = 2,
        min_detection_confidence=0.5
    ) as hands:
        results = hands.process(pose_image)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                pose_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
                )




In [10]:


#capture videofeed

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands = 2,
    min_detection_confidence=0.5
) as hands:
    while cap.isOpened():
        success,image = cap.read()
        if not success:
            print("skip empty camera frame")
        else:
            #pass by reference to boost performance
            image.flags.writeable = True
            results = hands.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                    )
            
            cv2.imshow('Mediapipe Hands', cv2.flip(image,1))
            #wait for kill signal 
            if cv2.waitKey(5) and 0xFF == 27:
                cv2.destroyAllWindows()
                break

cap.release()


KeyboardInterrupt: 

In [12]:

cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
) as pose:
    while cap.isOpened():
        success,image = cap.read()
        if not success:
            print("skip empty camera frame")
        else:
            #pass by reference to boost performance
            image.flags.writeable = True
            get_hands(image)
            results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
                )


            cv2.imshow('Mediapipe Hands', cv2.flip(image,1))
            #wait for kill signal 
            if cv2.waitKey(5) and 0xFF == 27:
                cv2.destroyAllWindows()
                break

cap.release()


AttributeError: 'NoneType' object has no attribute 'landmark'

In [9]:
#setup mediapipe
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode



options = HandLandmarkerOptions(
    base_options = BaseOptions(model_asset_path="./hand_landmarker.task"),
    running_mode = VisionRunningMode.IMAGE
)



In [14]:
mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)

In [21]:
with HandLandmarker.create_from_options(options) as landmarker:
    landmarker_result = landmarker.detect(mp_image)
    print(landmarker_result)

HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
